In [ ]:
# 필요한 라이브러리 설치
!pip install pandas numpy matplotlib scikit-learn folium geopy branca

# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import folium
from geopy.distance import geodesic
from google.colab import drive
from IPython.display import display, HTML


In [ ]:

# Google Colab을 사용하실 경우, 아래와 같이 데이터 파일을 업로드하시면 됩니다.
drive.mount("/content/drive/")

# 데이터 로딩
enrollment_data = pd.read_excel("/content/drive/MyDrive/송파구 초등학교 입학자_수정.xlsx")
geographical_data = pd.read_excel("/content/drive/MyDrive/강동송파교육지원청.xlsx")


Mounted at /content/drive/


In [ ]:

# 데이터 병합
merged_data = pd.merge(enrollment_data, geographical_data, on='학교명', how='inner')


In [ ]:

# 입학자 수 추세 계산
years = range(2008, 2024)
for year in years:
    if year + 1 in years:
        merged_data[f'Change_{year}-{year+1}'] = (merged_data[year + 1] - merged_data[year]) / merged_data[year]
merged_data['Mean_Change'] = merged_data[[f'Change_{year}-{year+1}' for year in years if year + 1 in years]].mean(axis=1)


In [ ]:

# 감소 추세의 학교 필터링
decreasing_enrollment_schools = merged_data[merged_data['Mean_Change'] < 0]


In [ ]:

# 클러스터링
coordinates = decreasing_enrollment_schools[['위도', '경도']]
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
cluster_labels = kmeans.fit_predict(coordinates)
decreasing_enrollment_schools['Cluster'] = cluster_labels

# 통폐합 후보군 학교 쌍
school_pairs = [
    ('서울방이초등학교', '서울방산초등학교'),
    ('서울가주초등학교', '서울가동초등학교'),
    ('서울토성초등학교', '서울풍납초등학교'),
    ('서울송전초등학교', '서울신천초등학교')
]

# 지도 시각화
map_center = [decreasing_enrollment_schools['위도'].mean(), decreasing_enrollment_schools['경도'].mean()]
school_map = folium.Map(location=map_center, zoom_start=12)
cluster_colors = ['red', 'blue', 'green', 'purple']

for idx, row in decreasing_enrollment_schools.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=5,
        popup=f"{row['학교명']} (Cluster {row['Cluster']})",
        color=cluster_colors[row['Cluster']],
        fill=True,
        fill_color=cluster_colors[row['Cluster']]
    ).add_to(school_map)

# 통폐합 후보군 학교 쌍에 대해 연결선 추가
for school1_name, school2_name in school_pairs:
    school1 = decreasing_enrollment_schools[decreasing_enrollment_schools['학교명'] == school1_name].iloc[0]
    school2 = decreasing_enrollment_schools[decreasing_enrollment_schools['학교명'] == school2_name].iloc[0]

    # Draw a line between the two schools
    folium.PolyLine(locations=[(school1['위도'], school1['경도']), (school2['위도'], school2['경도'])], color='gray').add_to(school_map)

# 지도 저장
school_map.save('/content/Schools_Cluster_Map.html')

# 파일을 다운로드할 수 있는 링크 제공
from google.colab import files
files.download('/content/Schools_Cluster_Map.html')

<ipython-input-6-ff6f8c95819c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decreasing_enrollment_schools['Cluster'] = cluster_labels


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>